In [164]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

import pandas as pd

# Scrape

In [165]:
def is_beer_entry(table_row):
    row_cells = table_row.findAll("td")
    beer_id = get_beer_id(row_cells[0].text)
    return (len(row_cells) == 8 and beer_id )

def get_beer_id(cell_value):
    r = re.match("^(\d{1,4})\.$", cell_value)
    if r and len(r.groups()) == 1:
        beer_id = r.group(1)
        return int(beer_id)
    else:
        return None
        

def get_all_beers(html_soup):
    beers_list = []
    all_rows_in_html_page = html_soup.findAll("tr")
    for table_row in all_rows_in_html_page:
        if is_beer_entry(table_row):
            row_cells = table_row.findAll("td")
            beer_entry = {
                "id": get_beer_id(row_cells[0].text),
                "name": row_cells[1].text,
                "brewery_name": row_cells[2].text,
                "brewery_location": row_cells[3].text,
                "style": row_cells[4].text,
                "size": row_cells[5].text,
                "abv": row_cells[6].text,    
                "ibu": row_cells[7].text
            }
            beers_list.append(beer_entry)
    return beers_list

In [166]:
html = urlopen("http://craftcans.com/db.php?search=all&sort=beerid&ord=desc&view=text")
html_soup = BeautifulSoup(html, 'html.parser')
beers_list = get_all_beers(html_soup)
df = pd.DataFrame(beers_list)
df.head(5)

,abv,brewery_location,brewery_name,ibu,id,name,size,style
0,4.5%,"Minneapolis, MN",NorthGate Brewing,50,2692,Get Together,16 oz.,American IPA
1,4.9%,"Minneapolis, MN",NorthGate Brewing,26,2691,Maggie's Leap,16 oz.,Milk / Sweet Stout
2,4.8%,"Minneapolis, MN",NorthGate Brewing,19,2690,Wall's End,16 oz.,English Brown Ale
3,6.0%,"Minneapolis, MN",NorthGate Brewing,38,2689,Pumpion,16 oz.,Pumpkin Ale
4,6.0%,"Minneapolis, MN",NorthGate Brewing,25,2688,Stronghold,16 oz.,American Porter


# Tidy Up

## Separate Observational Units: Beers, Breweries


* Problem: Beers and breweries are two different observational units.
* Solution: Create two tables. One for the beers and one for the breweries.

### Breweries

In [167]:
breweries = df[["brewery_location", "brewery_name"]]
breweries = breweries.drop_duplicates().reset_index(drop=True)
breweries["id"] = breweries.index
breweries.head(5)

,brewery_location,brewery_name,id
0,"Minneapolis, MN",NorthGate Brewing,0
1,"Louisville, KY",Against the Grain Brewery,1
2,"Framingham, MA",Jack's Abby Craft Lagers,2
3,"San Diego, CA",Mike Hess Brewing Company,3
4,"San Francisco, CA",Fort Point Beer Company,4


In [168]:
print(breweries.head(5).to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>brewery_location</th>
      <th>brewery_name</th>
      <th>id</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Minneapolis, MN</td>
      <td>NorthGate Brewing</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Louisville, KY</td>
      <td>Against the Grain Brewery</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Framingham, MA</td>
      <td>Jack's Abby Craft Lagers</td>
      <td>2</td>
    </tr>
    <tr>
      <th>3</th>
      <td>San Diego, CA</td>
      <td>Mike Hess Brewing Company</td>
      <td>3</td>
    </tr>
    <tr>
      <th>4</th>
      <td>San Francisco, CA</td>
      <td>Fort Point Beer Company</td>
      <td>4</td>
    </tr>
  </tbody>
</table>


### Beers

In [169]:
beers = pd.merge(df,
                 breweries,
                 left_on=["brewery_name", "brewery_location"],
                 right_on=["brewery_name", "brewery_location"],
                 sort=True,
                 suffixes=('_beer', '_brewery'))
beers = beers[["abv", "ibu", "id_beer",
               "name", "size", "style", "id_brewery"]]
beers_columns_rename = {
    "id_beer": "id",
    "id_brewery": "brewery_id"
}
beers.rename(inplace=True, columns=beers_columns_rename)
beers.head(5)

,abv,ibu,id,name,size,style,brewery_id
0,5.0%,N/A,1436,Pub Beer,12 oz.,American Pale Lager,408
1,6.6%,N/A,2265,Devil's Cup,12 oz.,American Pale Ale (APA),177
2,7.1%,N/A,2264,Rise of the Phoenix,12 oz.,American IPA,177
3,9.0%,N/A,2263,Sinister,12 oz.,American Double / Imperial IPA,177
4,7.5%,N/A,2262,Sex and Candy,12 oz.,American IPA,177


## Multiple variables in a single column

* Problem: The brewery location contains both the state and the city. These are 
two variables.
* Solution: Create two column. One for the state and one for the city.

In [170]:
breweries["city"] = breweries["brewery_location"].apply(
    lambda location: location.split(",")[0])
breweries["state"] = breweries["brewery_location"].apply(
    lambda location: location.split(",")[1])
breweries = breweries[["brewery_name", "city", "state", "id"]]
breweries.rename(inplace=True, columns={"brewery_name": "name"})
breweries.head(5)

,name,city,state,id
0,NorthGate Brewing,Minneapolis,MN,0
1,Against the Grain Brewery,Louisville,KY,1
2,Jack's Abby Craft Lagers,Framingham,MA,2
3,Mike Hess Brewing Company,San Diego,CA,3
4,Fort Point Beer Company,San Francisco,CA,4


In [171]:
print(breweries.head(5).to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>city</th>
      <th>state</th>
      <th>id</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>NorthGate Brewing</td>
      <td>Minneapolis</td>
      <td>MN</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Against the Grain Brewery</td>
      <td>Louisville</td>
      <td>KY</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Jack's Abby Craft Lagers</td>
      <td>Framingham</td>
      <td>MA</td>
      <td>2</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Mike Hess Brewing Company</td>
      <td>San Diego</td>
      <td>CA</td>
      <td>3</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Fort Point Beer Company</td>
      <td>San Francisco</td>
      <td>CA</td>
      <td>4</td>
    </tr>
  </tbody>
</table>


## Value is not in a valid format

* Problem: Alcohol by volume (`abv`) is stored as a string instead of a floating number.  
* Solution: Parse string and transform to a floating number

In [172]:
def string_pct_to_float(value):
    stripped_pct = str(value).strip('%')
    try:
        return float(stripped_pct)/100
    except ValueError:    
        return None

beers["abv"] = beers["abv"].apply(string_pct_to_float)

In [173]:
beers.head(5)

,abv,ibu,id,name,size,style,brewery_id
0,0.050,N/A,1436,Pub Beer,12 oz.,American Pale Lager,408
1,0.066,N/A,2265,Devil's Cup,12 oz.,American Pale Ale (APA),177
2,0.071,N/A,2264,Rise of the Phoenix,12 oz.,American IPA,177
3,0.090,N/A,2263,Sinister,12 oz.,American Double / Imperial IPA,177
4,0.075,N/A,2262,Sex and Candy,12 oz.,American IPA,177


## Value contain the measurement unit

- Problem: The size values are strings and they contain the measurement unit (ounce).
- Solution: Standardize all values to a numerical value of the same measurement unit.

In [175]:
for possible_value in set(beers["size"].tolist()):
    print(possible_value)

16 oz.
12 & 16 oz.
12 oz
19.2 oz.
16 oz
12 ounce
12 OZ.
16.9 oz.
12 oz.
24 oz.
32 oz.
12 oz. Slimline
19.2
8.4 oz.
24 oz. "Silo Can"
16 oz. Alumi-Tek®
16 oz. Alumi-Tek®	
12 oz. 


In [176]:
def extract_ounces(value):
    stripped = value.strip("oz")
    match = re.match("(\d{1,2}\.*\d*)", value)
    if match:
        return float(match.group(0))
    else:
        return None

beers["ounces"] = beers["size"].apply(extract_ounces)    
del beers["size"]
beers.head(5)

,abv,ibu,id,name,style,brewery_id,ounces
0,0.050,N/A,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,N/A,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,N/A,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,N/A,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,N/A,2262,Sex and Candy,American IPA,177,12.0


In [177]:
print(beers.head(5).to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>abv</th>
      <th>ibu</th>
      <th>id</th>
      <th>name</th>
      <th>style</th>
      <th>brewery_id</th>
      <th>ounces</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.050</td>
      <td>N/A</td>
      <td>1436</td>
      <td>Pub Beer</td>
      <td>American Pale Lager</td>
      <td>408</td>
      <td>12.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.066</td>
      <td>N/A</td>
      <td>2265</td>
      <td>Devil's Cup</td>
      <td>American Pale Ale (APA)</td>
      <td>177</td>
      <td>12.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.071</td>
      <td>N/A</td>
      <td>2264</td>
      <td>Rise of the Phoenix</td>
      <td>American IPA</td>
      <td>177</td>
      <td>12.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.090</td>
      <td>N/A</td>
      <td>2263</td>
      <td>Sinister</td>
      <td>American Dou

## Value contain "N/A" string

def string_to_int(value):
    try:
        return int(value)
    except ValueError:  
        return None

beers["ibu"] = beers["ibu"].apply(string_to_int)

## Save datasets to CSV

In [134]:
beers.to_csv("./data/beers.csv")
breweries.to_csv("./data/breweries.csv")

In [140]:
beers.head(10)

,abv,ibu,id,name,style,brewery_id,ounces
0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0
5,0.077,NaN,2261,Black Exodus,Oatmeal Stout,177,12.0
6,0.045,NaN,2260,Lake Street Express,American Pale Ale (APA),177,12.0
7,0.065,NaN,2259,Foreman,American Porter,177,12.0
8,0.055,NaN,2258,Jade,American Pale Ale (APA),177,12.0
9,0.086,NaN,2131,Cone Crusher,American Double / Imperial IPA,177,12.0


In [142]:
breweries.head(10)

,name,city,state
0,NorthGate Brewing,Minneapolis,MN
1,Against the Grain Brewery,Louisville,KY
2,Jack's Abby Craft Lagers,Framingham,MA
3,Mike Hess Brewing Company,San Diego,CA
4,Fort Point Beer Company,San Francisco,CA
5,COAST Brewing Company,Charleston,SC
6,Great Divide Brewing Company,Denver,CO
7,Tapistry Brewing,Bridgman,MI
8,Big Lake Brewing,Holland,MI
9,The Mitten Brewing Company,Grand Rapids,MI
